In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, cross_val_predict
from scipy.interpolate import griddata

In [ ]:
# Load data from the .mat file
data = sio.loadmat('../data/ML_Optimal_method.mat')
eddy_double_LES = data['eddy_double_LES']
eddy_double_TBNN = data['eddy_double_TBNN']
Sij = data['Sij']
Rij = data['Rij']
x = data['x']
y = data['y']
x_point = data['x_point']
y_point = data['y_point']
bij_LES = data['bij_LES']
bij_TBNN = data['bij_TBNN']

### edd viscosity = function(Sij, Rij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij an Rij as features
X = np.concatenate((Sij[:, [0, 1, 3, 4]], Rij[:, [0, 1, 3, 4]]), axis=1)
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define parameter grid for MLPRegressor hyperparameters
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],  # Regularization term
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [1000]
}

# Initialize an MLPRegressor
mlp = MLPRegressor(solver='adam', random_state=42)

# Perform GridSearchCV with 5-fold cross-validation to find best hyperparameters
kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(mlp, param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_scaled, Y)
best_params = grid_search.best_params_
best_rmse = np.sqrt(-grid_search.best_score_)
print("Best hyperparameters from GridSearchCV:", best_params)
print("Best RMSE from GridSearchCV:", best_rmse)

# Initialize final MLPRegressor with the best hyperparameters
final_mlp = MLPRegressor(solver='adam', random_state=42, **best_params)

# Evaluate the model using 5-fold cross-validation
cv_scores = cross_val_score(final_mlp, X_scaled, Y, cv=kf, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-np.mean(cv_scores))
print("MLPRegressor Cross-validated RMSE:", cv_rmse)

# Get cross-validated predictions
Y_pred_cv = cross_val_predict(final_mlp, X_scaled, Y, cv=kf)
print("MLPRegressor Cross-validated R2:", r2_score(Y, Y_pred_cv))

# Train the final MLPRegressor on the full dataset
final_mlp.fit(X_scaled, Y)
Y_pred = final_mlp.predict(X_scaled)
print("MLPRegressor Training RMSE:", np.sqrt(mean_squared_error(Y, Y_pred)))
print("MLPRegressor Training R2:", r2_score(Y, Y_pred))

# Plot predictions vs. actual values
plt.figure(figsize=(10, 6))
plt.plot(Y_LES, label='Actual eddy_double_LES', color='green')
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred, label='MLP Prediction (Train)', color='red', linestyle='--')
plt.plot(Y_pred_cv, label='MLP Prediction (CV)', color='orange', linestyle='-.')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('MLPRegressor Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('MLPRegressor Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()

### edd viscosity = function(Sij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij as features
X = Sij[:, [0, 1, 3, 4]]
# Use eddy_double_TBNN as the target variable, and eddy_double_LES as a reference
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define parameter grid for MLPRegressor hyperparameters
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],  # Regularization term
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [1000]
}

# Initialize an MLPRegressor
mlp = MLPRegressor(solver='adam', random_state=42)

# Perform GridSearchCV with 5-fold cross-validation to find best hyperparameters
kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(mlp, param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_scaled, Y)
best_params = grid_search.best_params_
best_rmse = np.sqrt(-grid_search.best_score_)
print("Best hyperparameters from GridSearchCV:", best_params)
print("Best RMSE from GridSearchCV:", best_rmse)

# Initialize final MLPRegressor with the best hyperparameters
final_mlp = MLPRegressor(solver='adam', random_state=42, **best_params)

# Evaluate the model using 5-fold cross-validation
cv_scores = cross_val_score(final_mlp, X_scaled, Y, cv=kf, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-np.mean(cv_scores))
print("MLPRegressor Cross-validated RMSE:", cv_rmse)

# Get cross-validated predictions
Y_pred_cv = cross_val_predict(final_mlp, X_scaled, Y, cv=kf)
print("MLPRegressor Cross-validated R2:", r2_score(Y, Y_pred_cv))

# Train the final MLPRegressor on the full dataset
final_mlp.fit(X_scaled, Y)
Y_pred = final_mlp.predict(X_scaled)
print("MLPRegressor Training RMSE:", np.sqrt(mean_squared_error(Y, Y_pred)))
print("MLPRegressor Training R2:", r2_score(Y, Y_pred))

# Plot predictions vs. actual values
plt.figure(figsize=(10, 6))
plt.plot(Y_LES, label='Actual eddy_double_LES', color='green')
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred, label='MLP Prediction (Train)', color='red', linestyle='--')
plt.plot(Y_pred_cv, label='MLP Prediction (CV)', color='orange', linestyle='-.')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('MLPRegressor Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('MLPRegressor Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()

### edd viscosity = function(Sij, bij)

In [ ]:
# Prepare features and target variables
# Use columns 0, 1, 3, and 4 from Sij as features and bij_LES as additional features
X = np.concatenate((Sij[:, [0, 1, 3, 4]], bij_TBNN[:, [0, 1, 3, 4]]), axis=1)
# Use eddy_double_TBNN as the target variable, and eddy_double_LES as a reference
Y = eddy_double_TBNN.ravel()
Y_LES = eddy_double_LES.ravel()

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define parameter grid for MLPRegressor hyperparameters
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],  # Regularization term
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [1000]
}

# Initialize an MLPRegressor
mlp = MLPRegressor(solver='adam', random_state=42)

# Perform GridSearchCV with 5-fold cross-validation to find best hyperparameters
kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(mlp, param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_scaled, Y)
best_params = grid_search.best_params_
best_rmse = np.sqrt(-grid_search.best_score_)
print("Best hyperparameters from GridSearchCV:", best_params)
print("Best RMSE from GridSearchCV:", best_rmse)

# Initialize final MLPRegressor with the best hyperparameters
final_mlp = MLPRegressor(solver='adam', random_state=42, **best_params)

# Evaluate the model using 5-fold cross-validation
cv_scores = cross_val_score(final_mlp, X_scaled, Y, cv=kf, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-np.mean(cv_scores))
print("MLPRegressor Cross-validated RMSE:", cv_rmse)

# Get cross-validated predictions
Y_pred_cv = cross_val_predict(final_mlp, X_scaled, Y, cv=kf)
print("MLPRegressor Cross-validated R2:", r2_score(Y, Y_pred_cv))

# Train the final MLPRegressor on the full dataset
final_mlp.fit(X_scaled, Y)
Y_pred = final_mlp.predict(X_scaled)
print("MLPRegressor Training RMSE:", np.sqrt(mean_squared_error(Y, Y_pred)))
print("MLPRegressor Training R2:", r2_score(Y, Y_pred))

# Plot predictions vs. actual values
plt.figure(figsize=(10, 6))
plt.plot(Y_LES, label='Actual eddy_double_LES', color='green')
plt.plot(Y, label='Actual eddy_double_TBNN', color='blue')
plt.plot(Y_pred, label='MLP Prediction (Train)', color='red', linestyle='--')
plt.plot(Y_pred_cv, label='MLP Prediction (CV)', color='orange', linestyle='-.')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.title('MLPRegressor Prediction vs. Actual')
plt.legend()
plt.grid(True)
plt.show()

# Interpolate the results onto the spatial grid
points = np.column_stack((x_point.ravel(), y_point.ravel()))
eddy_double_LES_interp = griddata(points, eddy_double_LES.ravel(), (x, y), method='cubic').reshape(x.shape)
eddy_double_TBNN_interp = griddata(points, eddy_double_TBNN.ravel(), (x, y), method='cubic').reshape(x.shape)
Y_pred_interp = griddata(points, Y_pred.ravel(), (x, y), method='cubic').reshape(x.shape)

# Set common color scale limits for the spatial plots
vmin = -0.05
vmax = 0.20

# Plot the interpolated maps in a vertical stack (3 subplots)
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
im0 = axs[0].pcolormesh(x, y, eddy_double_LES_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[0].set_title('eddy double LES')
axs[0].set_aspect('equal', adjustable='box')
axs[0].set_xlim(np.min(x), np.max(x))
axs[0].set_ylim(np.min(y), np.max(y))

im1 = axs[1].pcolormesh(x, y, eddy_double_TBNN_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[1].set_title('eddy double TBNN')
axs[1].set_aspect('equal', adjustable='box')
axs[1].set_xlim(np.min(x), np.max(x))
axs[1].set_ylim(np.min(y), np.max(y))

im2 = axs[2].pcolormesh(x, y, Y_pred_interp, shading='auto', cmap='jet', vmin=vmin, vmax=vmax)
axs[2].set_title('MLPRegressor Y pred')
axs[2].set_aspect('equal', adjustable='box')
axs[2].set_xlim(np.min(x), np.max(x))
axs[2].set_ylim(np.min(y), np.max(y))

fig.subplots_adjust(right=0.85, hspace=0.1)
cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7])
fig.colorbar(im2, cax=cbar_ax)
plt.show()